In [23]:
!pip install gradio -q

In [24]:
import gradio as gr
import random
import pandas as pd

def optimasi_hybrid_dashboard(lahan, pupuk, modal, p_padi, k_padi, b_padi, p_jagung, k_jagung, b_jagung, p_sing, k_sing, b_sing):
    # --- 1. AREA DATA KOMODITAS (Reference Data dari Input Dashboard) ---
    # Data ini diambil dari inputan user di kolom tengah (bisa di-update per 3 bulan)
    data = {
        'Padi':     {'profit': p_padi, 'pupuk': k_padi, 'biaya': b_padi},
        'Jagung':   {'profit': p_jagung, 'pupuk': k_jagung, 'biaya': b_jagung},
        'Singkong': {'profit': p_sing, 'pupuk': k_sing, 'biaya': b_sing}
    }

    # --- 2. AREA PERHITUNGAN 1: GENETIC ALGORITHM (10 Iterasi) ---
    populasi = []
    for i in range(1, 11):
        # Menggenerate nilai acak lahan (x1, x2, x3) sesuai prinsip GA
        x1 = random.uniform(0, lahan)
        x2 = random.uniform(0, lahan - x1)
        x3 = random.uniform(0, lahan - x1 - x2)

        # Perhitungan total profit, pupuk, dan biaya berdasarkan inputan Reference Data
        prof = (x1 * data['Padi']['profit']) + (x2 * data['Jagung']['profit']) + (x3 * data['Singkong']['profit'])
        pupk = (x1 * data['Padi']['pupuk']) + (x2 * data['Jagung']['pupuk']) + (x3 * data['Singkong']['pupuk'])
        biay = (x1 * data['Padi']['biaya']) + (x2 * data['Jagung']['biaya']) + (x3 * data['Singkong']['biaya'])

        # Cek Batasan (Constraints) berdasarkan Modal dan Stok Pupuk
        status = "AMAN" if (pupk <= pupuk and biay <= modal) else "OVER"

        populasi.append({
            "It": i, "Padi": round(x1,2), "Jagung": round(x2,2),
            "Singkong": round(x3,2), "Profit": int(prof),
            "Biaya": int(biay), "Status": status
        })

    # Konversi hasil populasi ke DataFrame untuk ditampilkan di Tabel
    df = pd.DataFrame(populasi)
    df_aman = df[df['Status'] == "AMAN"]

    # Jika tidak ada satupun iterasi yang AMAN
    if df_aman.empty:
        return "❌ GAGAL: Modal atau Pupuk kurang!", df, "0", "0", "0", "N/A"

    # --- 3. LOGIKA PENYOROTAN (Highlighting Baris Terbaik) ---
    # Mencari iterasi dengan profit tertinggi dari status yang AMAN
    best_idx = df_aman['Profit'].idxmax()
    row = df.iloc[best_idx]

    # Memberikan warna hijau pada baris terbaik di tabel GA
    styled_df = df.style.apply(lambda r: ['background-color: #d1f7d1; color: black; font-weight: bold' if r.name == best_idx else '' for _ in r], axis=1)

    # --- 4. AREA PERHITUNGAN 2: GRG/GRADIENT (Local Optimization) ---
    # Melakukan fine-tuning (perbaikan tipis) dari hasil terbaik GA untuk memaksimalkan lahan
    curr_x1, curr_x2, curr_x3 = row['Padi'], row['Jagung'], row['Singkong']
    step = 0.05
    for _ in range(100):
        new_x1 = curr_x1 + step
        n_pupuk = (new_x1 * data['Padi']['pupuk']) + (curr_x2 * data['Jagung']['pupuk']) + (curr_x3 * data['Singkong']['pupuk'])
        n_biaya = (new_x1 * data['Padi']['biaya']) + (curr_x2 * data['Jagung']['biaya']) + (curr_x3 * data['Singkong']['biaya'])

        if (new_x1 + curr_x2 + curr_x3 <= lahan) and (n_pupuk <= pupuk) and (n_biaya <= modal):
            curr_x1 = new_x1
        else:
            break

    # --- 5. AREA HASIL AKHIR (Decision Support Summary) ---
    f_profit = (curr_x1 * data['Padi']['profit']) + (curr_x2 * data['Jagung']['profit']) + (curr_x3 * data['Singkong']['profit'])
    f_biaya  = (curr_x1 * data['Padi']['biaya']) + (curr_x2 * data['Jagung']['biaya']) + (curr_x3 * data['Singkong']['biaya'])
    f_pupuk  = (curr_x1 * data['Padi']['pupuk']) + (curr_x2 * data['Jagung']['pupuk']) + (curr_x3 * data['Singkong']['pupuk'])

    res_status = f"✅ Sukses: Iterasi {int(row['It'])} terpilih & dioptimalkan dengan GRG."
    res_detail = (f"💰 Estimasi Profit: Rp {int(f_profit):,}\n"
                  f"💸 Total Biaya: Rp {int(f_biaya):,}\n"
                  f"🧪 Pupuk Terpakai: {f_pupuk:,.1f} Kg")

    return res_status, styled_df, f"{curr_x1:.2f} Ha", f"{curr_x2:.2f} Ha", f"{curr_x3:.2f} Ha", res_detail

# --- ANTARMUKA DASHBOARD (Gradio UI Layout) ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌱 Smart Farm Optimizer (Hybrid GA-GRG)")
    gr.Markdown("Optimasi keuntungan menggunakan metode **Hybrid GA-GRG** dengan data dinamis.")

    with gr.Row():
        # Kolom 1: Input Operasional (Lahan, Pupuk, Modal)
        with gr.Column():
            gr.Markdown("### 📥 1. Input Operasional")
            l_in = gr.Number(label="Total Lahan (Ha)", value=100)
            p_in = gr.Number(label="Stok Pupuk (Kg)", value=5000)
            m_in = gr.Number(label="Modal (Rp)", value=500000000)

        # Kolom 2: Reference Data Komoditas (Input harga/biaya per 3 bulan)
        with gr.Column():
            gr.Markdown("### ⚙️ 2. Reference Data (Update per 3 Bulan)")
            with gr.Accordion("Setting Padi", open=True):
                with gr.Row():
                    p_prof = gr.Number(label="Profit/Ha", value=15000000)
                    p_pupk = gr.Number(label="Pupuk/Ha", value=60)
                    p_biay = gr.Number(label="Biaya/Ha", value=4000000)
            with gr.Accordion("Setting Jagung", open=True):
                with gr.Row():
                    j_prof = gr.Number(label="Profit/Ha", value=12000000)
                    j_pupk = gr.Number(label="Pupuk/Ha", value=40)
                    j_biay = gr.Number(label="Biaya/Ha", value=3000000)
            with gr.Accordion("Setting Singkong", open=True):
                with gr.Row():
                    s_prof = gr.Number(label="Profit/Ha", value=10000000)
                    s_pupk = gr.Number(label="Pupuk/Ha", value=30)
                    s_biay = gr.Number(label="Biaya/Ha", value=2000000)

    # Tombol Aksi
    btn = gr.Button("🚀 JALANKAN OPTIMASI", variant="primary")

    with gr.Row():

        # Kolom 3: Tabel Iterasi
        with gr.Column():
            gr.Markdown("### 📑 3. Tabel Perhitungan (GA Iterations)")
            tabel = gr.DataFrame()
        # Kolom 4: Output Rekomendasi
        with gr.Column():
            gr.Markdown("### 📊 4. Hasil Rekomendasi (Summary)")
            status_txt = gr.Textbox(label="Status Sistem", interactive=False)
            with gr.Row():
                o_padi = gr.Label(label="Padi")
                o_jagu = gr.Label(label="Jagung")
                o_sing = gr.Label(label="Singkong")
            o_det = gr.Textbox(label="Financial & Inventory Detail", lines=4)



    # Logika Trigger Klik Tombol
    btn.click(optimasi_hybrid_dashboard,
              inputs=[l_in, p_in, m_in, p_prof, p_pupk, p_biay, j_prof, j_pupk, j_biay, s_prof, s_pupk, s_biay],
              outputs=[status_txt, tabel, o_padi, o_jagu, o_sing, o_det])

# Launch Program 2
demo.launch()

/tmp/ipython-input-1419808256.py:79: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ccbc8ba7cfa3ce86b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
